In [1]:
import openai
import os
import json
import requests
from dotenv import load_dotenv

load_dotenv()

client = openai.OpenAI()
MODEL = "gpt-4o-mini"


## 2. Tool 정의: get_today_weather 함수 및 JSON 스키마


In [3]:
# === Tool: 오늘 날씨 조회 함수 ===
def get_today_weather(city: str) -> str:
    """
    지정된 도시의 현재 날씨 정보를 가져옵니다.
    
    Args:
        city: 도시 이름 (예: "Seoul", "Busan", "Tokyo")
    
    Returns:
        날씨 정보를 담은 JSON 문자열 (예: {"weather": "Clear", "temp": 25.5})
    """
    API_KEY = "cc8369ec59d8c647f4797cec0fc9fef1"
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={API_KEY}&units=metric&lang=kr"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        # 필요한 정보 추출
        weather_main = data.get("weather", [{}])[0].get("main", "Unknown")
        weather_description = data.get("weather", [{}])[0].get("description", "Unknown")
        temp = data.get("main", {}).get("temp")
        feels_like = data.get("main", {}).get("feels_like")
        humidity = data.get("main", {}).get("humidity")
        
        # JSON 문자열로 반환
        result = {
            "weather": weather_main,
            "description": weather_description,
            "temp": temp,
            "feels_like": feels_like,
            "humidity": humidity
        }
        return json.dumps(result, ensure_ascii=False)
    except requests.exceptions.HTTPError as e:
        if response.status_code == 404:
            return json.dumps({"error": f"'{city}' 도시를 찾을 수 없습니다."}, ensure_ascii=False)
        else:
            return json.dumps({"error": f"API 요청 중 에러 발생: {e}"}, ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"API 요청 중 에러 발생: {e}"}, ensure_ascii=False)

# === JSON 스키마 정의 ===
weather_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_today_weather",
        "description": "지정된 도시의 현재 날씨 정보(날씨 상태, 온도, 체감 온도, 습도 등)를 조회합니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨를 조회할 도시 이름 (예: Seoul, Busan, Tokyo, New York)"
                }
            },
            "required": ["city"]
        }
    }
}

# === 사용 가능한 함수를 매핑 ===
available_functions = {
    "get_today_weather": get_today_weather
}


## 3. 함수 직접 테스트

먼저 `get_today_weather` 함수가 올바르게 동작하는지 확인해봅니다.


In [4]:
# get_today_weather 함수 직접 테스트
print("=== get_today_weather 함수 직접 테스트 ===")
test_result = get_today_weather("Seoul")
print(f"서울 날씨 결과: {test_result}")

# JSON 파싱하여 보기 좋게 출력
parsed_result = json.loads(test_result)
print("\n파싱된 결과:")
for key, value in parsed_result.items():
    print(f"  {key}: {value}")


=== get_today_weather 함수 직접 테스트 ===
서울 날씨 결과: {"weather": "Clear", "description": "맑음", "temp": 5.76, "feels_like": 4.19, "humidity": 87}

파싱된 결과:
  weather: Clear
  description: 맑음
  temp: 5.76
  feels_like: 4.19
  humidity: 87


## 4. Agent Full-Loop 구현: "오늘 서울 날씨 어때?"

Agent의 전체 동작 사이클을 단계별로 구현합니다:
1. LLM에 Tool 호출 요청
2. 결정된 Tool을 실제로 실행
3. Tool 실행 결과를 바탕으로 최종 답변 생성


In [5]:
# 사용자 질문
messages = [{"role": "user", "content": "오늘 서울 날씨 어때?"}]
tools = [weather_tool_schema]

print("💬 1단계: LLM에 Tool 호출 요청")
response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
response_message = response.choices[0].message
messages.append(response_message)

tool_calls = response_message.tool_calls
if tool_calls:
    print("✅ LLM이 Tool 사용을 결정했습니다.")
    print("\n🔧 2단계: 결정된 Tool을 실제로 실행")
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        print(f"- 실행 함수: {function_name}({', '.join([f'{k}={v}' for k, v in function_args.items()])})")

        function_response = function_to_call(**function_args)
        print(f"- 실행 결과: {function_response}")

        messages.append({"tool_call_id": tool_call.id, "role": "tool", "name": function_name, "content": function_response})

    print("\n💭 3단계: Tool 실행 결과를 바탕으로 최종 답변 생성")
    final_response = client.chat.completions.create(model=MODEL, messages=messages)
    print("\n===== Agent의 최종 답변 =====")
    weather_agent_answer = final_response.choices[0].message.content
    print(weather_agent_answer)
else:
    print("❌ LLM이 Tool 사용을 결정하지 않았습니다.")
    final_response = client.chat.completions.create(model=MODEL, messages=messages)
    print("\n===== Agent의 최종 답변 =====")
    print(final_response.choices[0].message.content)


💬 1단계: LLM에 Tool 호출 요청
✅ LLM이 Tool 사용을 결정했습니다.

🔧 2단계: 결정된 Tool을 실제로 실행
- 실행 함수: get_today_weather(city=Seoul)
- 실행 결과: {"weather": "Clear", "description": "맑음", "temp": 5.76, "feels_like": 4.19, "humidity": 87}

💭 3단계: Tool 실행 결과를 바탕으로 최종 답변 생성

===== Agent의 최종 답변 =====
오늘 서울의 날씨는 맑고, 기온은 약 5.8도입니다. 체감 온도는 4.2도이며, 습도는 87%입니다. 외출 시 따뜻하게 입고 나가세요!


In [ ]:
# 다른 도시들의 날씨도 테스트해보기
test_cities = ["Seoul", "Busan", "Tokyo", "New York"]

for city in test_cities:
    
    messages = [{"role": "user", "content": f"오늘 {city} 날씨 어때?"}]
    tools = [weather_tool_schema]
    
    response = client.chat.completions.create(model=MODEL, messages=messages, tools=tools, tool_choice="auto")
    response_message = response.choices[0].message
    messages.append(response_message)
    
    tool_calls = response_message.tool_calls
    if tool_calls:
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            function_response = function_to_call(**function_args)
            messages.append({"tool_call_id": tool_call.id, "role": "tool", "name": function_name, "content": function_response})
        
        final_response = client.chat.completions.create(model=MODEL, messages=messages)
        print(final_response.choices[0].message.content)
    else:
        print("Tool이 호출되지 않았습니다.")



🌍 Seoul 날씨 조회
오늘 서울의 날씨는 맑고, 기온은 약 5.8도입니다. 체감 온도는 4.2도이며, 습도는 87%입니다. 따뜻하게 입으세요!

🌍 Busan 날씨 조회
오늘 부산의 날씨는 맑고, 기온은 약 9도이며, 체감 온도는 약 8도입니다. 습도는 81%입니다. 좋은 하루 되세요!

🌍 Tokyo 날씨 조회
오늘 도쿄의 날씨는 맑습니다. 현재 온도는 13.7도이며, 체감 온도는 12.85도입니다. 습도는 66%입니다. 좋은 하루 되세요!

🌍 New York 날씨 조회
오늘 뉴욕의 날씨는 맑습니다. 현재 기온은 약 4.25도이며, 체감 온도는 1.95도입니다. 습도는 68%입니다. 외출하시기 전에 따뜻하게 입으세요!
